In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama.chat_models import ChatOllama
from langchain.tools.render import render_text_description
from langgraph.prebuilt import create_react_agent
from langchain.output_parsers import PydanticOutputParser

In [ ]:
import operator
import uuid
import json
from json import JSONDecodeError
from pydantic import BaseModel, Field, ValidationError, field_validator
from utility import generate_tools, extract_relevant_info
from typing import TypedDict, Annotated, Sequence, List, Literal, ClassVar

In [ ]:
_, model_tools = generate_tools("resume.json")
model_name = "deepseek-r1:14b"
job_link = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4037021039&discover=recommended&discoveryOrigin=JOBS_HOME_JYMBII"
# job_link = "https://www.linkedin.com/jobs/view/4246320795"

In [ ]:
class ToolCall(BaseModel):
    _id_counter: ClassVar[int] = 1

    tool_name: Literal['navigate_browser', 'query_resume', 'get_all_elements', 'fill_element', 'click_element'] \
        = Field(description="Name of the tool to be used.")
    tool_inputs: dict
    tool_id: int

    def __init__(self, **data):
        data['tool_id'] = self.__class__._id_counter
        self.__class__._id_counter += 1

        super().__init__(**data)

    def __restart_toolCounter(self):
        self.__class__._id_counter = 0

In [ ]:
class Supervisor_Response(BaseModel):
    application_success: bool
    tool_call: List[ToolCall]
    reasoning: str

    @field_validator('tool_call', mode='before')
    @classmethod
    def check_tool_call_keys(cls, v):
        if not isinstance(v, list):
            raise ValueError('tool_call must be a list')
        for item in v:
            if not isinstance(item, dict):
                raise ValueError('Each tool call must be a dict')
            required_keys = {'tool_name', 'tool_inputs'}
            if not required_keys.issubset(item.keys()):
                missing = required_keys - item.keys()
                raise ValueError(f"Missing keys in tool call: {missing}. "
                                 "Each tool call must use 'tool_name' and 'inputs' as keys.")
        return v

    @field_validator('application_success', mode='before')
    @classmethod
    def check_application_success(cls, v):
        if not isinstance(v, bool):
            raise ValueError('application_success must be boolean')
        return v

In [ ]:
class AgentState(TypedDict):
    job_link: str
    messages: Annotated[Sequence[HumanMessage | AIMessage | SystemMessage | ToolMessage], operator.add]
    application_success: bool
    tool_calls: List[ToolCall]
    tool_outputs: List[dict]
    tools_output_summary: str


In [ ]:
reasoning_model_prompt_template = """
You are a Job Application Supervisor agent whose sole responsibility is to select the next tool action at each step of an automated job application process by analyzing the provided tool outputs summary.

General Instructions:
- Choose exactly one tool_call per turn.
- Analyze only the **tools_output_summary** to decide the next action.
- If **tools_output_summary** is empty, begin with `navigate_browser`.
- If **tools_output_summary** does not contain elements information use `get_all_elements`.
- Set "application_success": True only after the final submission step.

Specific Instructions:
- Login pages:
  1. Use `query_resume` with input `login` if credentials are needed.
  2. Fill "username" then "password" via `fill_element`.
  3. Click "Sign In" via `click_element`.
- General application forms:
  1. If fields are unknown, call `get_all_elements`.
  2. Use `query_resume` to retrieve relevant information.
  3. Fill each required field in order via `fill_element`.
  4. Click “Next” or “Continue” or "Submit" via `click_element`.
- Submission pages:
  - Click “Submit” via `click_element`, then set "application_success": True.

Available Tools and Descriptions:
{tools_description}

Generate output in the following format:
{format_instructions}
"""

In [ ]:
# reasoning_model_prompt_template = """
# You are a Job Application Supervisor agent.

# Your task: At each step, Analyse current tool_output summary to select the next tool_call and its inputs.

# General Instructions:
# 1. At the begining of the application always navigate to the job link.
# 2. Read tool descriptions to understand what should be the input for the tool.
# 3. never call more than one tool in a single step.
# 4. Analyse the tool_output summary to select next action. Provide reasoning for the same.
# 5. Only interact with essential elements on any page.
# 6. Only once the Job Application is completed return 'application_flag' as True.

# Specific Instruction:
# 1. Always use 'query_resume' tool with appropriate keywords (refer tool_description) to get information e.g. ('login' for login credentials etc).
# 2. At the begining of the application, The tool_call summary is expected to be empty.
# 2. For general Job application pages the element priority is as follows:
#    - Required form fields
#    - Continue / Next buttons
#    - Submit buttons
# 3. For login pages the element priority is as follows:
#    - First fill username
#    - Then fill password
#    - Finally click Sign In

# Available Tools and Descriptions:
# {tools_description}

# Generate output in the following format:
# {format_instructions}
# """

In [ ]:
model_tools_dictionary = {
        'navigate_browser': model_tools[4],
        'query_resume': model_tools[3],
        'get_all_elements': model_tools[2],
        'fill_element': model_tools[1],
        'click_element': model_tools[0]
        }

In [ ]:
supervisor_parser = PydanticOutputParser(pydantic_object=Supervisor_Response)

supervisor_formatted_prompt = ChatPromptTemplate.from_messages([
    ("system", reasoning_model_prompt_template),
    MessagesPlaceholder(variable_name="messages"),
    (
        "human",
        """Job_link: {job_link}\nTool Call History: {tools_output_summary}\nSelect next action from {options}"""
    )
]).partial(
    tools_description = render_text_description(model_tools),
    format_instructions = supervisor_parser.get_format_instructions(),
    options = ", ".join(['navigate_browser', 'query_resume', 'get_all_elements', 'fill_element', 'click_element'])
    )

In [ ]:
reason_llama = ChatOllama(model=model_name)
reasoning_chain = supervisor_formatted_prompt | reason_llama.with_structured_output(Supervisor_Response)

In [ ]:
def prepare_state(state: dict):
    tool_outputs = state.get("tool_outputs", [])
    summary = ""

    for output in tool_outputs:
        tool_id = output.get('tool_id', None)
        if tool_id:
            del output['tool_id']
        summary += "\n\n".join([f"{key} - {tool_id if tool_id else -1}\nContent: {val}" for key, val in output.items()])

    return {**state, "tools_output_summary": summary}

In [ ]:
async def Apply_job(state: AgentState) -> AgentState:
    while state.get('application_success') != True:
        prepared_state = prepare_state(state)

        supervisor_response = reasoning_chain.invoke(prepared_state)

        supervisor_result = Supervisor_Response.model_validate(supervisor_response)

        flag = supervisor_result.application_success

        tool_call_list = supervisor_result.tool_call

        assert isinstance(tool_call_list, list)

        reason = supervisor_result.reasoning
        print("Reasoning: ", reason)

        # Not required checks as model_validate will throw error if not proper output
        if isinstance(flag, bool) and flag is not None:
            state['application_success'] = flag

        if tool_call_list:
            state['tool_calls'].extend(tool_call_list)


        tool_call = tool_call_list[0]
        name = tool_call.tool_name
        inputs = tool_call.tool_inputs
        call_id = tool_call.tool_id

        response = ''
        print("Tool Call", name, inputs)

        actual_tool = model_tools_dictionary.get(name)

        try:
            response = json.loads(await actual_tool.arun(inputs))
        except JSONDecodeError as e:
            print(f"Error {e}")

        result = response.get('result', {})

        if not result:
            state['messages'].append(HumanMessage(content=f"Tool Call for tool: {name} did not return output. Try Again", name="Tool_Call"))
            state['tool_outputs'].append({name: "Tool Call did not return output. Try Again", 'tool_id': call_id})

        if result and isinstance(result, str):
            state['tool_outputs'].append({name: result, 'tool_id': call_id})
        else:
            assert isinstance(result, dict) == True
            title = result.get('Page Title', '')
            content = result.get('Page Elements Summary', '')
            state['tool_outputs'].append({name: title, 'tool_id': call_id})
            state['tool_outputs'].append({name: content, 'tool_id': call_id})

        state['messages'].append(HumanMessage(content=json.dumps(result), name="Tool_Call"))


In [ ]:
start_state = AgentState(
    job_link = job_link,
    messages=[
        HumanMessage(content="Begin Job Application.")
    ],
    application_flag=False,
    tool_calls=[],
    tool_outputs=[],
    tools_output_summary=""
)

In [ ]:
end_state = await Apply_job(start_state)

In [ ]:
test = reasoning_chain.invoke(start_state)
test

In [ ]:
test.tool_call[0]

In [ ]:
job_agent = create_react_agent(
    model=ollama_llm,
    tools=model_tools,
    # prompt=system_prompt.format(tools=render_text_description(model_tools)),
    state_schema=AgentState
)

In [ ]:
user_prompt = """
For the job_link: {job_link}

Use the provided tools appropriately to complete the job application.

Available Tools and Descriptions:
{tools}

""".format(job_link=job_link, tools=render_text_description(model_tools))

In [ ]:
initial_state = dict(
    # elements_summary={},
    # elementsOfInterest={},
    # actions_planned={},
    messages=[HumanMessage(content=user_prompt)],
    remaining_steps=20
)

In [ ]:
async for chunk in job_agent.astream(
    initial_state,
    stream_mode="updates"
):
    # print(await extract_relevant_info(chunk))
    print(chunk)

In [ ]:
# [{'type': 'AIMessage', 'content': "To answer your question, I will follow these steps:\n\n1. Navigate to the job link.\n2. Retrieve Page title and elements summary.\n3. Analyze the elements summary along with page title to determine the elements of interest.\n4. For Identified elements of interest containing action 'fill', Retrieve text with resume query tool.\n5. Use the appropriate tool to fill or click the elements of interest along with the relevant inputs.\n\nHere is a JSON that best answers your prompt:\n\n```json", 'tool_calls': [{'tool_name': 'navigate_browser', 'input': {'url': 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4037021039&discover=recommended&discoveryOrigin=JOBS_HOME_JYMBII'}}, {'tool_name': 'get_all_elements', 'input': {}}]}]
# [{'type': 'ToolMessage', 'content': {'Page Title': '', 'Page Elements Summary': []}, 'tool_name': 'get_all_elements'}]
# [{'type': 'ToolMessage', 'content': 'Navigating successfull.', 'tool_name': 'navigate_browser'}]
# [{'type': 'AIMessage', 'content': "We can't get the page elements summary since it's empty. Let's navigate to the page and try again.\n\n\n", 'tool_calls': [{'tool_name': 'get_all_elements', 'input': {}}]}]
# [{'type': 'ToolMessage', 'content': {'Page Title': 'LinkedIn Login, Sign in | LinkedIn', 'Page Elements Summary': [{'selector': 'button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.sign-in-with-apple-button', 'attributes': {'id': None, 'aria-label': 'Sign in with Apple', 'type': 'button', 'maxlength': None}, 'possible_actions': ['click']}, {'selector': '#username', 'attributes': {'id': 'username', 'aria-label': 'Email or phone', 'type': 'email', 'maxlength': None}, 'possible_actions': ['fill']}, {'selector': '#password', 'attributes': {'id': 'password', 'aria-label': 'Password', 'type': 'password', 'maxlength': None}, 'possible_actions': ['fill']}, {'selector': 'button.btn__primary--large.from__button--floating', 'attributes': {'id': None, 'aria-label': 'Sign in', 'type': 'submit', 'maxlength': None}, 'possible_actions': ['click']}, {'selector': '#btn-resend-otp', 'attributes': {'id': 'btn-resend-otp', 'aria-label': 'Resend email', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': '#otp-cancel-button', 'attributes': {'id': 'otp-cancel-button', 'aria-label': 'Back', 'type': None, 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__button', 'attributes': {'id': None, 'aria-label': None, 'type': None, 'maxlength': None}, 'possible_actions': ['click']}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'العربية (Arabic) 1 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'বাংলা (Bangla) 2 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Čeština (Czech) 3 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Dansk (Danish) 4 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Deutsch (German) 5 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Ελληνικά (Greek) 6 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link.language-selector__link--selected', 'attributes': {'id': None, 'aria-label': 'English (English) 7 of 36 selected', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Español (Spanish) 8 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'فارسی (Persian) 9 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Suomi (Finnish) 10 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Français (French) 11 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'हिंदी (Hindi) 12 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Magyar (Hungarian) 13 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Bahasa Indonesia (Indonesian) 14 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Italiano (Italian) 15 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'עברית (Hebrew) 16 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '日本語 (Japanese) 17 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '한국어 (Korean) 18 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'मराठी (Marathi) 19 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Bahasa Malaysia (Malay) 20 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Nederlands (Dutch) 21 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Norsk (Norwegian) 22 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'ਪੰਜਾਬੀ (Punjabi) 23 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Polski (Polish) 24 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Português (Portuguese) 25 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Română (Romanian) 26 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Русский (Russian) 27 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Svenska (Swedish) 28 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'తెలుగు (Telugu) 29 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'ภาษาไทย (Thai) 30 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Tagalog (Tagalog) 31 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Türkçe (Turkish) 32 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Українська (Ukrainian) 33 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': 'Tiếng Việt (Vietnamese) 34 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '简体中文 (Chinese (Simplified)) 35 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}, {'selector': 'button.language-selector__link', 'attributes': {'id': None, 'aria-label': '正體中文 (Chinese (Traditional)) 36 of 36 ', 'type': 'button', 'maxlength': None}, 'possible_actions': []}]}, 'tool_name': 'get_all_elements'}]
# [{'type': 'AIMessage', 'content': 'Elements of interest: \n1. username input field\n2. password input field\n3. login button\n\nAction Plan:\n1. Fill username input field\n2. Fill password input field\n3. Click on the login button\n\nTool call to fill username input field:\n\n', 'tool_calls': [{'tool_name': 'fill_element', 'input': {'selector': '#username', 'text': 'linkedin email'}}]}]
# [{'type': 'ToolMessage', 'content': 'Filled element #username with text linkedin email', 'tool_name': 'fill_element'}]
# [{'type': 'AIMessage', 'content': '', 'tool_calls': [{'tool_name': 'fill_element', 'input': {'selector': '#password', 'text': 'linkedin password'}}]}]
# [{'type': 'ToolMessage', 'content': 'Filled element #password with text linkedin password', 'tool_name': 'fill_element'}]
# [{'type': 'AIMessage', 'content': '', 'tool_calls': [{'tool_name': 'click_element', 'input': {'selector': 'button.btn__primary--large.from__button--floating'}}]}]
# [{'type': 'ToolMessage', 'content': "Clicked element 'button.btn__primary--large.from__button--floating'", 'tool_name': 'click_element'}]
# [{'type': 'AIMessage', 'content': 'elements_summary = [{"selector": "button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.artdeco-global-alert-action.artdeco-button.artdeco-button--inverse.artdeco-button--2.artdeco-button--primary", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.sign-in-with-apple-button", "attributes": {"id": null, "aria-label": "Sign in with Apple", "type": "button", "maxlength": null}, "possible_actions": ["click"]}, {"selector": "#username", "attributes": {"id": "username", "aria-label": "Email or phone", "type": "email", "maxlength": null}, "possible_actions": []}, {"selector": "#password", "attributes": {"id": "password", "aria-label": "Password", "type": "password", "maxlength": null}, "possible_actions": []}, {"selector": "button.btn__primary--large.from__button--floating", "attributes": {"id": null, "aria-label": "Sign in", "type": "submit", "maxlength": null}, "possible_actions": ["click"]}, {"selector": "#btn-resend-otp", "attributes": {"id": "btn-resend-otp", "aria-label": "Resend email", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "#otp-cancel-button", "attributes": {"id": "otp-cancel-button", "aria-label": "Back", "type": null, "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__button", "attributes": {"id": null, "aria-label": null, "type": null, "maxlength": null}, "possible_actions": ["click"]}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "العربية (Arabic) 1 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "বাংলা (Bangla) 2 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Čeština (Czech) 3 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Dansk (Danish) 4 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Deutsch (German) 5 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Ελληνικά (Greek) 6 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link.language-selector__link--selected", "attributes": {"id": null, "aria-label": "English (English) 7 of 36 selected", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Español (Spanish) 8 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "मराठी (Marathi) 19 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Bahasa Malaysia (Malay) 20 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Nederlands (Dutch) 21 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Norsk (Norwegian) 22 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "ਪੰਜਾਬੀ (Punjabi) 23 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Polski (Polish) 24 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Português (Portuguese) 25 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Română (Romanian) 26 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Русский (Russian) 27 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Svenska (Swedish) 28 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "తెలుగు (Telugu) 29 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "ภาษาไทย (Thai) 30 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Tagalog (Tagalog) 31 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Türkçe (Turkish) 32 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Українська (Ukrainian) 33 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "Tiếng Việt (Vietnamese) 34 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "简体中文 (Chinese (Simplified)) 35 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}, {"selector": "button.language-selector__link", "attributes": {"id": null, "aria-label": "正體中文 (Chinese (Traditional)) 36 of 36 ", "type": "button", "maxlength": null}, "possible_actions": []}]\n\nWe are now on the LinkedIn login page.', 'tool_calls': []}]

In [ ]:
# How to use request library to chat with locally served ollama model.
# https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion
# https://github.com/ollama/ollama
# import requests

# url = "http://localhost:11434/api/generate"
# data = {
#     "model": "gemma3:12b-it-qat",
#     "prompt": "Why is the sky blue?",
#     "stream": False
# }

# response = requests.post(url, json=data)
# print(response.text)
